In [1]:
import findspark 
findspark.init()

In [2]:
%load_ext memory_profiler
import pyspark.pandas as ps
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from tqdm import notebook
import warnings
warnings.filterwarnings(action='ignore')
# station 정보
station = pd.read_csv('D:\git_local_repository\data_for_project\seoul_bike\seoul_bike_station_01_12.csv',encoding='CP949')
seoul_bike = pd.read_parquet('D:\git_local_repository\data_for_project\seoul_bike\seoul_bike_2021.parquet.gzip')

#plotly continuous_color
continuous_color = pd.read_csv('data/plotly_continuous_color_codes.csv')


## 생활권 분석
### 가설
1. 해당 대여소가 몰리는 곳인지 퍼지는 곳인지 분류할 수 있음
    - 몰리는 곳에 대한 정의는 ? 다양한 대여소에서 온다. 나가는 따릉이보다 들어가는 따릉이가 더 많다.
    
    - 지역구간 단방향 이동인지 양방향 이동인지 파악 할 수 있음. 어느정도 범위는 양방향이고 음수이면 왼쪽방향 양수이면 오른쪽 방향이라는 걸 알 수 있겠지? 
    - 몰리는 곳 설정 + 흐름을 합치면 뭔가 재밌는 그림이 나올거 같다는 생각이다.
    => 기상 흐름도를 채용하면 되지 않을까?

<br>

1. 어디가 생활반경인지 보다는 물리적으로 지역임에도 심리가 먼 지역이 어디인지를 보는데 초점을 맞춤. 
    - 뭐가 됐던 사람들이 가지는 근처 지역에 대한 심리가 실제로 얼추 맞는지 살펴보면 되지 않을까?
    
    - 심리가 먼 지역에 대한 정의는? 물리적으로 가까움에도 top 20에 포함되지 않은 지역 (top10,top20,top50 다 생각해보자)
    - 20분~ 30분 거리를 등고선으로 하면 활동 범위를 알 수 있으려나? 아무리 가까워도 안가는 곳이라면 20-30분 거리로 넓혀봤자 포함안된다.

<br>


1. 따릉이로 안가본 서울 도심 체험하기
    - 본인이 주로 애용하는 대여소를 기점으로 사람들이 자주 가는 순으로 여행지를 택할 수 있게 하자.
    - 많은 사람들이 이용한다는 말은 그만큼 따릉이로 이동하기 어렵지 않다는 말
  
    - 방법

        1.원하는 따릉이 대여소 검색

        2.table로 선택 되면 그롷게 하고 안되면 드롭다운 방식으로 보여주기

        3.선택하면 collector로 올라가게 함

        4.추천 여행지 우측 테리블에 정렬
        
        5.클릭하면 해당 지역으로 이동

<br>


1. 따릉이로 최적 거리 이동하기
    - 내가 A대여소에서 어느 지점으로 간다고 지역을 누르면 반경에 있는 따릉이 대여소 몇 군데를 골라 예상 운행 시간을 알려준다.
      
      사람들이 가장 많이 이용한 대여소 또는 최단 거리가 걸리는 대여소를 알려주면 될 듯
      
    - 많은 사람들이 이용한다는 말은 그만큼 따릉이로 이동하기 어렵지 않다는 말
    - 지도앱 예상 시간과 따릉이 이용 예상시간 차이는 매우 크다. 
  
    -  따릉이로 가기 좋은 길인지, 얼마정도 시간이 걸리는지 예상용도
        네이버 지도 예상시간은 따릉이를 이용하기엔 너무 빠르다.
    - 네이버 지도와 데이터를 바탕으로 최소 이동시간이 소요되는 지역을 추천할 수 있다.

<br>


### 필요한 사항 정리
1. 몰리는 곳 정의
예시) (여의나루 역 2번출구 => 다른 지역) 총 id 개수 와  (다른 지역 => 여의나루 역 2번 출구) 총 id 개수 비교 

    & (여의나루 2번 출구 => 다른 지역으로 이동한 따릉이 대여 개수) 와 (다른 지역에서 여의나루 2번 출구로 이동한 따릉이 개수) 비교

    * 그러려면 st_id2가 여의나루인 경우 id 개수, 따릉이 대여 대수가 필요하고 st_id1이 여의나루인 상태에서 전 지역으로 이동한 개수와 따릉이 대여 개수가 필요하겠네

    * 개수만 새는거니까 그렇게 오래 걸리지 않겠지?

    * 이때 10개,20개,30개 이상 데이터가 있는 경우 비교 하는 것도 좋고 아니면 전체 비율에서 0.005% 이상인 비율만 찾아도 될거 같다는 생각이네

<br>


2. 대여소간 이동 흐름 분석해보기 
예시)  월촌초 <=> 신목동역을 예로들면 월촌초 => 신목동 가는 횟수와 신목동 => 월촌초 가는 횟수를 비교해서 $\frac{월=>신 - 신=>월}{양방향 개수}$한다음 

    +- 0.1 내외면 양방향이고 양수면 월=> 신 방향이 많은거고 음수면 신=> 월 방향이 많은거고  0.2는 약간의 추세 0.5는 추세 0.8은 압도적 추세 이런식으로 만들면 될 것 같음.

    * top 10,20,50개 등 다양한 기준을 가지고 흐름을 만들어 보는 것도 좋을듯 = 아니면 빈도수를 30개 50개 이상 등으로 설정하면 다양한 흐름이 나오지 않을까 생각이 든다.

    * 계산은 대여소 번호 낮은것 - 높은 것으로 원칙을 정하고 한 곳에만 값을 매겨놓으면 되지 않을까? 아니면 아예 st_id1 -> st_id2 항목을 새로 만들어서 흐름만 관리해도 좋고

    * 기상 흐름도가 도움이 될까? 잘 맞는게 뭐가 있는지 한 번 봐보자. 아니면 화살표로 해봐도 될거같다. 대여소 주위에서 그 지역까지 화살표로 나가는걸로 하고 흐름이 쎈것만 opacity를 높게 주면 되지 않을까?

    * 화살표로 한다면 일단 지역 하나를 클릭하면 그 지역을 중심으로 어떻게 흐름이 발생하는지 보면 재밌을듯 만약 퍼지는 대여소라면 나가는 방향이 들어가는 방향보다 훨씬 많겠지 

    *  나가는 방향 빨간색 들어오는 방향 파란색으로 하면 뭔가 재밌을것 같다.


<br>


1. 둘이 합치고 꾸미면 뭔가 흐름을 볼 수 있지 않을까?

<br>


4. 따릉이로 심리적인 생활반경 그리기
예시) 예전에 주 활동 무대는 염창역 인근이었다면 지금 주 활동 무대는 신목동역이다. 

    사실 염창역도 그렇게 멀지 않지만 산이라는 자연 장애물 때문에 심리적으로 멀게 느껴진다. 

      다른 지역도 분명 그럴진데 따릉이 대여소가 어디를 가장 많이 가는지(top 20,30,50 등)를 분석하면 알 것 같다.

    * 개별 반경 100m < x <5km 따릉이 대여소를 개별 대여소 별로 찾는다. st_id_1과 st_id=2가 같으니 이것도 따로 표를 새로 만들면 될 것 같다.
    
    * 따릉이 이용소 top 20,50 등을 구하고 색을 다르게 하면 표현 될 것 같다.

    * density plot을 그릭 서울 전역을 확대하면 어느정도 생활권이 나오려나?

    * 위 방법 보다는 본인이 가장 애용하는 따릉이 대여소를 클릭하면 관련 생활권이 나오게 하는거 어떨까?

    * 많은 사람들에게도 물어봐보자 이스밍한테도 물어봐보자.


In [3]:
# #불러오기
# seoul_bike_1_2 = pd.read_hdf('D:\git_local_repository\data_for_project\seoul_bike\preprocessed_bike_2021.hdf5')
# seoul_bike_2_2 = pd.read_hdf('D:\git_local_repository\data_for_project\seoul_bike\preprocessed_bike_2_2.hdf5')

# seoul_bike = pd.concat([seoul_bike_1_2,seoul_bike_2_2])

# # 1분당 이동 거리(m)
# seoul_bike['m_pm'] = (seoul_bike['dist'] / (seoul_bike['riding_time'])).round(decimals=0)  # 1분당 거리 계산

# #한국에는 자전거에 미친놈들이 많은건가 아니면 오류인가
# # 선수급도 50km/h 속도가 나온다. 1분에 830미터 정도 달린다고 한다.
# # 동호회 수준에서도 30km/h면 수준급 실력이라 한다. 1분에 500m 정도 된다.
# # 4745948, 도심 한가운데서 분당 1000m의 속도로 54키로를 달리는게 말이되나.. 54km면 여의도 공원 => 잠원 한강공원을 왕복하고도 다시 출발해 절반은 더 가야된다.
# # 이게 말이되나.. 

# # 결론
# # 1분당 거리 계산 (1분당 400미터 이상인 경우 error로 간주)

# seoul_bike = seoul_bike.query('m_pm <= 400')

# seoul_bike


### 아래부터 테스트용 

In [4]:
station = pd.read_csv('D:\git_local_repository\data_for_project\seoul_bike\seoul_bike_station_01_12.csv',encoding='CP949',index_col=0)
station.rename(columns={'대여소 번호' : 'id'}, inplace=True)
wanting_ids = [207]
min_time=50
max_time=120
min_count=50
sorted_st_id2=seoul_bike[seoul_bike['st_id1'].isin(wanting_ids)]['st_id2'].value_counts() # 해당 지역에서 이동하는 지역 소팅
sorted_st_id2= sorted_st_id2.drop(index=wanting_ids)
min_counts=np.where(sorted_st_id2.to_numpy() >= min_count)[0][-1]
print(f'검색기준 : 해당 대여소와 최소 {min_count}번 이상 대여기록이 있는 대여소 검색')
print(f'총 {min_counts}개 대여소 확보')

seoul_bike

result_station = []
for station_id in notebook.tqdm(sorted_st_id2[:int(min_counts)].index) : 
    # BM = (seoul_bike['st_id1'].isin(wanting_ids)) & ((seoul_bike['st_id2'].isin([station_id])))
    # filter_id = seoul_bike[BM]['riding_time'].value_counts().sort_values(ascending=False)
    BM = (seoul_bike['st_id1'] == wanting_ids[0]) & (seoul_bike['st_id2'] == station_id)
    filter_id = seoul_bike[BM]['riding_time'].value_counts().sort_values(ascending=False)
    try :
        mean = filter_id.index[:5].values.mean()
        percent = round(sum(filter_id[:5].values)/sum(filter_id),2)
        num = sum(filter_id[:5])
        total_num = sum(filter_id)
        result_station.append([station_id,mean,percent,num,total_num])
    except :
        pass
sorted_ids = pd.DataFrame(result_station)


검색기준 : 해당 대여소와 최소 50번 이상 대여기록이 있는 대여소 검색
총 298개 대여소 확보


  0%|          | 0/298 [00:00<?, ?it/s]

KeyboardInterrupt: 

**isin이 query보다 빠름**

In [ ]:
%%timeit
wanting_ids = 207 ; station_id = 754
BM = (seoul_bike['st_id1'].isin([wanting_ids])) & ((seoul_bike['st_id2'].isin([station_id])))
filter_id = seoul_bike[BM]['riding_time'].value_counts().sort_values(ascending=False)

127 ms ± 9.95 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
wanting_ids = 207 ; station_id = 754
filter_id = seoul_bike.query('st_id1 == @wanting_ids & st_id2 == @station_id')['riding_time'].value_counts().sort_values(ascending=False)

217 ms ± 6.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
wanting_ids = 207 ; station_id = 754
BM = (seoul_bike['st_id1'] == wanting_ids) & (seoul_bike['st_id2'] == station_id)
filter_id = seoul_bike[BM]['riding_time'].value_counts().sort_values(ascending=False)

99.5 ms ± 9.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Row 개수 줄이는 식을 먼저 써야 속도가 빨라진다.**

In [ ]:
# %%timeit
# wanting_ids = 754
# sorted_st_id2=seoul_bike[seoul_bike['st_id1'] == wanting_ids]['st_id2'].value_counts() # 해당 지역에서 이동하는 지역 소팅

44.1 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# %%timeit
# wanting_ids = 754
# sorted_st_id2=seoul_bike['st_id2'][seoul_bike['st_id1'] == wanting_ids].value_counts() # 해당 지역에서 이동하는 지역 소팅

62.1 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 위에부터 테스트용

### 대여소별 따릉이 여행지 추천

In [28]:
def searching_location(wanting_ids,min_time=1,max_time=60,min_count=50) : 
    station = pd.read_csv('D:\git_local_repository\data_for_project\seoul_bike\seoul_bike_station_01_12.csv',encoding='CP949',index_col=0)
    station.rename(columns={'대여소 번호' : 'id'}, inplace=True)
    sorted_st_id2=seoul_bike.query('st_id1 == @wanting_ids')['st_id2'].value_counts() # 해당 지역에서 이동하는 지역 소팅
    sorted_st_id2=seoul_bike[seoul_bike['st_id1'] == wanting_ids]['st_id2'].value_counts() # 해당 지역에서 이동하는 지역 소팅

    sorted_st_id2= sorted_st_id2.drop(index=wanting_ids)
    min_counts=np.where(sorted_st_id2 >= min_count)[0][-1]
    print(f'검색기준 : 해당 대여소와 최소 {min_count}번 이상 대여기록이 있는 대여소 검색')
    print(f'총 {min_counts}개 대여소 확보')
    result_station = []
    for station_id in notebook.tqdm(sorted_st_id2[:min_counts].index) : 
        # 가장 단순하지만 loop에서 9.21 it/s로 제일빠름
        BM = (seoul_bike['st_id1'] == wanting_ids) & (seoul_bike['st_id2'] == station_id)
        filter_id = seoul_bike[BM]['riding_time'].value_counts().sort_values(ascending=False)
        
        ## 여러 값을 한 번에 계산한다는 장점이 있지만 상대적으로 느림 Loop 7.44 it/s
        # BM = (seoul_bike['st_id1'].isin([wanting_ids])) & ((seoul_bike['st_id2'].isin([station_id])))
        # filter_id = seoul_bike['riding_time'][BM].value_counts().sort_values(ascending=False)

        # 시각적으로 직관적이고 간단하게 작성할 수 있음. 그러나 Loop 3.21 it/s 임. 심한 경우 1.44 it/s인 경우도 발생
        # filter_id = seoul_bike.query('st_id1 == @wanting_ids & st_id2 == @station_id')['riding_time'].value_counts().sort_values(ascending=False)
        try :
            mean = filter_id.index[:3].values.mean()
            percent = round(sum(filter_id[:3].values)/sum(filter_id),2)
            num = sum(filter_id[:3])
            total_num = sum(filter_id)
            result_station.append([station_id,mean,percent,num,total_num])
        except :
            pass
    
    sorted_ids = pd.DataFrame(result_station,columns=['id','est_time','percent','num','total_num']).query('num > 10 & @min_time <= est_time <= @max_time')
    sorted_ids.columns = ['id', '예상도착시간', '비율','예상시간 내 도착 횟수' ,'대여소 이용 이력']
    return pd.merge(sorted_ids, station, how='left', on='id').sort_values(by='예상도착시간',ascending=False)


# trip_id = 207
# result_test = searching_location(trip_id, min_time=15, max_time=60, min_count=50)

# 10.66it/s 1083.75mb
# 7.28it/s 1101.96mb
# 5.12it/s 1099.29mib

peak memory: 918.60 MiB, increment: 0.00 MiB


**대여소 추천 그래프**

In [30]:
trip_id = 754
result_test = searching_location(trip_id, min_time=15, max_time=60, min_count=50)

### 그래프 그리기 
# 지역 순위 표시
fig=px.scatter_mapbox(result_test.sort_values(by='대여소 이용 이력')[:10].dropna(), 
                        lat="위도",lon="경도",
                        hover_name="주소2",
                        hover_data=['예상도착시간','비율','예상시간 내 도착 횟수','대여소 이용 이력','id'],
                        color='예상도착시간',opacity=0.6,
                        # size='예상도착시간',
                        size='대여소 이용 이력',size_max=20,
                        # color_discrete_map = {'해당 대여소로 향하는 지역': 'navy', '이동량이 비슷한 지역': 'palegreen', '해당 대여소에서 주로 가는 지역': 'burlywood'},
                        zoom=10,height=500,)



# 해당 따릉이 대여소 색 표시
fig_3 = go.Figure(go.Scattermapbox(
        name='location',
        lat=station.query('id==@trip_id')['위도'],
        lon=station.query('id==@trip_id')['경도'],
        mode='markers',
        # marker = {'size': 20, 'symbol': "bus"},
        marker=go.scattermapbox.Marker(size=10,color='black',opacity=0.8),
        text=station.query('id==@trip_id')['주소2'].values,
    ))                            
fig.add_trace(fig_3.data[0])
fig.update_layout(mapbox_style="carto-positron")



검색기준 : 해당 대여소와 최소 50번 이상 대여기록이 있는 대여소 검색
총 34개 대여소 확보


  0%|          | 0/34 [00:00<?, ?it/s]

In [ ]:
# ### 대여소 별 거리 측정
# asd = station.query('id ==[744,766]')
# haversine_np(asd.iloc[0,-3],asd.iloc[0,-2],asd.iloc[1,-3],asd.iloc[1,-2])
# # station[station['주소2'].str.contains("월촌중")]
# # station.query('id == 744')

NameError: name 'haversine_np' is not defined

In [18]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return m
    
def id_map(id,n, dist=1000, min_count = 500) :
    
    total_num_left = seoul_bike[seoul_bike['st_id1']== id]['st_id2'].value_counts() # 현재 대여소에서 빌리면 주로 향하는 목적지는?
    total_num_left = total_num_left[~total_num_left.index.isin([id])] # 같은 곳에서 빌려서 같은 곳에 반납하는 경우 제거
    total_num_left = total_num_left[: np.where(total_num_left > n)[0][-1]] # 최소 빌린 횟수 

    # 2 to 1 다른 대여소 => 여의나루 
    total_num_right = seoul_bike[seoul_bike['st_id2']==id]['st_id1'].value_counts() # 어느 대여소에서 현재 대여소로 이동할까?
    total_num_right = total_num_right[total_num_right.index != id] # 같은 곳에서 빌려서 같은 곳에 반납하는 경우 제거
    total_num_right = total_num_right[: np.where(total_num_right > n)[0][-1]] # 최소 빌린 횟수

    # concat
    result_concat = pd.concat([total_num_left,total_num_right],axis=1)
    result_concat.fillna((n/2),inplace=True)
    result_concat.rename(columns=dict(st_id2='1to2',st_id1='2to1'),inplace=True) 
    result_concat['rate'] = round((result_concat["1to2"] / (result_concat["1to2"]+result_concat["2to1"])),4)
    result_concat['diff'] = abs((result_concat["1to2"] - result_concat["2to1"])) # 얼마나 차이나는지 
    result_concat['total'] = abs((result_concat["1to2"] + result_concat["2to1"])) # 얼마나 차이나는지 
    result_concat['category'] = pd.cut(result_concat['rate'], bins=[0,0.45,0.55,result_concat['rate'].max()],
                                                              labels=["해당 대여소로 향하는 지역", "이동량이 비슷한 지역" ,"해당 대여소에서 주로 가는 지역"])
    add_info_result = pd.merge(result_concat,station, how='left',left_on=result_concat.index,right_on='id') # station 정보 추가
    
    add_info_result.loc[:,'diff'] = add_info_result['diff'] + 100  # 망할 plotly가 min_size를 구현 안해놔서 이렇게라도 해서 min을 키워야함.

    ## 해당 대여소 100미터 이내 따릉이 기록 제거(fig_1에 해당)
    add_info_result['dist'] = haversine_np(station.query('id==@id')['경도'].values,station.query('id==@id')['위도'].values,add_info_result['경도'].values,add_info_result['위도'].values) # 거리측정
    add_info_result = add_info_result.query('dist > 100')

    # 해당 대여소 1km 반경 내 따릉이 대여소 표시(fig_2에 해당)
    station_haversine = haversine_np(station.query('id==@id')['경도'].values,station.query('id==@id')['위도'].values,station['경도'].values,station['위도'].values)
    station_haversine = pd.concat([station,pd.Series(station_haversine)],axis=1)
    station_haversine.rename(columns={0:'dist'},inplace=True)
    station_haversine = station_haversine.query('@dist >= dist > 0')

    # return add_info_result




    ### 그래프 그리기 
    # 반경 내 따릉이 대여소 표현
    fig_2 = go.Figure(go.Scattermapbox(
        name= f'station with in {round(dist/1000,2)}km',
        lat=station_haversine['위도'],
        lon=station_haversine['경도'],
        mode='markers',
        # marker = {'size': 20, 'symbol': "bus"},
        marker=go.scattermapbox.Marker(size=8,color='black',opacity=0.2),
        text=station_haversine['주소2'],
        ))                          

    # 지역 순위 표시
    fig=px.scatter_mapbox(add_info_result.query(' total > @min_count'), #category != "neutral" & 
                            lat="위도",lon="경도",
                            hover_name="주소2",
                            hover_data=['1to2','2to1','id'],
                            color='category',opacity=0.7,
                            size='diff', size_max=20,
                            color_discrete_map = {'해당 대여소로 향하는 지역': 'midnightblue', 
                                                  '이동량이 비슷한 지역': 'hotpink',
                                                  '해당 대여소에서 주로 가는 지역': 'coral'},
                            zoom=12,height=500,)

    fig.add_trace(fig_2.data[0])


    # 해당 따릉이 대여소 색 표시
    fig_3 = go.Figure(go.Scattermapbox(
            name='location',
            lat=station.query('id==@id')['위도'],
            lon=station.query('id==@id')['경도'],
            mode='markers',
            # marker = {'size': 20, 'symbol': "bus"},
            marker=go.scattermapbox.Marker(size=10,color='fuchsia',opacity=0.8),
            text=station.query('id==@id')['주소2'].values,
        ))                            
    fig.add_trace(fig_3.data[0])
    fig.update_layout(mapbox_style="carto-positron")
    return fig

id= 473
n = 50

id_map(id, n, min_count=500)

In [17]:
print(station[station['주소2'].str.contains('을지로4가')][['주소2','id']])

             주소2   id
241  을지로4가역 1번출구  389
316  을지로4가역 9번출구  473


In [ ]:
color_code = pd.read_csv('data/plotly_color_codes.csv')
print(color_code['0'].to_list())

['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond', 'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen', 'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue', 'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite', 'gold', 'goldenrod', 'gray', 'grey', 'green', 'greenyellow', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgray', 'lightgrey', 'lightgreen', 'lightpink', 'lightsalmon', 'lightseagreen', 

### 2. 대여소간 흐름 분석 먼저 해보자 나가고 들어오는 그런 흐름을 한 번 만들어보고 싶다.

### 실수로 실행시킬까봐 # 처리해놓았음

중요하니 지우지 말자

In [ ]:
# # id : 대여소 id   n : 대여소간 최소 대여수 ex) 여의나루 => 1000여개 대여소 중 n회 이상 이동한 대여소만 포함
# def con_dis(id,n) :
#     try : 
#         # 1 to 2 여의나루 => 다른 대여소
#         total_num_left = seoul_bike[seoul_bike['st_id1']==id]['st_id2'].value_counts() # 현재 대여소에서 빌리면 주로 향하는 목적지는?
#         total_num_left = total_num_left[~total_num_left.index.isin([id])] # 같은 곳에서 빌려서 같은 곳에 반납하는 경우 제거

#         # 2 to 1 다른 대여소 => 여의나루 
#         total_num_right = seoul_bike[seoul_bike['st_id2']==id]['st_id1'].value_counts() # 어느 대여소에서 현재 대여소로 이동할까?
#         total_num_right = total_num_right[total_num_right.index != id] # 같은 곳에서 빌려서 같은 곳에 반납하는 경우 제거

#         a = total_num_left.sum()
#         b = total_num_right.sum()

#         return [id,a,b,round(foo(a,b),3)]

#     except :
#         return [id]

# def foo(x,y):
#     try:
#         return x/y
#     except ZeroDivisionError:
#         return 0


# id1 = seoul_bike['st_id2'].value_counts()
# id2 = seoul_bike['st_id1'].value_counts()
# id1 = id1[id1>500].index # 대여횟수 최소 500회 이상인 대여소 선정
# id2 = id2[id2>500].index
# total_id=np.unique(np.append(id1,id2))

# n = 50 # 대여소간 이동횟수 최소 n회
# result_sum=[]
# for num,ids in notebook.tqdm(enumerate(total_id))  :
#     result_con_dis = con_dis(ids,n) # con_dis 함수 
#     result_sum.append(result_con_dis)



In [ ]:
asd = pd.merge(pd.DataFrame(result_sum),station,how='left',left_on=0,right_on='id').sort_values(by=2,ascending=False)
asd.columns = ['id_du','1to2','2to1','portion', 'id', '주소2', '주소1', '지역', '위도', '경도', '설치일']
asd.drop(columns=['id_du','주소1','설치일','portion'],inplace=True)
asd.dropna(inplace=True)
asd['rate'] = asd['1to2'] / (asd['1to2'] + asd['2to1'])
asd['category'] = pd.cut(asd['rate'], bins=[0,0.45,0.55,asd['rate'].max()],labels=["몰리는 지역", "neutral" ,"퍼지는 지역"])
asd['diff'] = abs((asd["1to2"] - asd["2to1"])) # 얼마나 차이나는지 
asd.to_csv('con_dis.csv',encoding='CP949')

In [ ]:
asd = pd.read_csv('con_dis.csv',encoding='CP949')

In [ ]:
fig=px.scatter_mapbox(asd.query('category != "neutral" & diff > 3000'), lat="위도", lon="경도", hover_name="주소2", hover_data=['1to2','2to1','id'], color='category', size='diff',size_max=20, color_continuous_scale='ylorrd', zoom=10, height=500, opacity=0.6)

# 해당 따릉이 대여소 색 표시하기
# fig_2 = go.Figure(go.Scattermapbox(
#         lat=station.query('id==@id')['위도'],
#         lon=station.query('id==@id')['경도'],
#         mode='markers',
#         # marker = {'size': 20, 'symbol': "bus"},
#         marker=go.scattermapbox.Marker(size=10,color='purple'),
#         text=[station.query('id==@id')['주소2']],
#     ))                            
# fig.add_trace(fig_2.data[0])
fig.update_layout(mapbox_style="carto-positron")                                                              